In [2]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import regex as re
from tqdm import tqdm

import urllib.request
from bs4 import BeautifulSoup
import requests
import json as json
import datetime


from selenium.webdriver.support.ui import Select

In [3]:
data = pd.read_csv("전국대회_명단.csv", index_col = None)
data['시작일'] = pd.to_datetime(data['시작일'])


cut = datetime.date(2019,10,31)
valid_data = data[data['시작일'].dt.date < cut]

#print(valid_data)
t_code = valid_data['대회번호']
t_code

#create dataframe and scrape

columns = ['대회명','종별', '경기결과']
combined_df = pd.DataFrame(columns = columns)
    

for i in tqdm(range(60, len(valid_data))):
    
    t_num = t_code.iloc[i]
    url = r"https://result.sports.or.kr/sq/S11.jsp?classCd=41&toCd=" + str(t_num) + "&kindCd=&baseClassCd=&gameDate=&page=1"
    driver2 = webdriver.Chrome(executable_path=r'C:\Users\h00ns\Power Ranking\power_rank\chromedriver')
    driver2.get(url)
    
    
    t_row = valid_data.loc[valid_data['대회번호'] == int(t_num)]
    t_name = t_row.iloc[0].at['대회명']
    print(t_name)

    #time.sleep(5)

    #종목구분 알아내기

    soup = BeautifulSoup(driver2.page_source, 'lxml')
    scroll_tabs = soup.find_all(class_ = 'serch_list')
    divisions = scroll_tabs[0].find("select", {"id": "kindCd"}).contents
    div_list = []
    
    tourney_dic = {}
    for div in divisions:
        string = div.string
        div_list.append(string)
    
    
    for div in div_list:
        select = Select(driver2.find_element_by_id("kindCd"))
        select.select_by_visible_text(str(div))

        search_btn = driver2.find_element_by_xpath("//*[@id='selectDiv']/table/tbody/tr/td[4]/a")
        search_btn.click()

        soup2 = BeautifulSoup(driver2.page_source, 'lxml')
        a = soup2.find_all('a', href=True)
        match_link_list = []

        
        division = []
        matches = []
        tournament = []

        for links in a:
            href = links.get('href')
            keyword = re.compile('^T02')

            if re.search(keyword, str(href)):
                link = "https://result.sports.or.kr/sq/"+ str(href)
                match_link_list.append(link)

                division.append(str(div))
                matches.append(link)
                tournament.append(t_name)


        tourney_dic[str(div)] = match_link_list
    
    columns = ['대회명','종별', '경기결과']
    match_df = pd.DataFrame(columns = columns)
    tournament = []
    section = []
    all_links = []

    for key in tourney_dic:
        value = tourney_dic[str(key)]
        for site in value:
            tournament.append(t_name)
            section.append(key)
            all_links.append(site)

    match_df['종별'] = section
    match_df['경기결과'] = all_links
    match_df['대회명'] = tournament
    
    combined_df = combined_df.append(match_df)
    combined_df.to_csv('All_Match_Links_4.csv')

combined_df.to_csv('All_Match_Links_4.csv')

  0%|          | 0/5 [00:06<?, ?it/s]

제91회 전국체육대회


UnexpectedAlertPresentException: Alert Text: 선택한 대회정보를 찾을 수 없습니다.
Message: unexpected alert open: {Alert text : 선택한 대회정보를 찾을 수 없습니다.}
  (Session info: chrome=89.0.4389.90)


In [6]:
t_num = t_code.iloc[10]
url = r"https://result.sports.or.kr/sq/S11.jsp?classCd=41&toCd=" + str(t_num) + "&kindCd=&baseClassCd=&gameDate=&page=1"
driver2 = webdriver.Chrome(executable_path=r'C:\Users\h00ns\Power Ranking\power_rank\chromedriver')
driver2.get(url)